# Translation of non-Dutch corpora

* MIMIC III
* MIMIC III CXR
* MIMIC IV
* eICU
* ApolloCorpus
* Meditron guidelines

Translate using:
* NLLB, 
* MariaMT, 
* DeepL, 
* Google Translate, 
* ChatGPT, 
* Claude, 
* Gemini

In [11]:
import datasets
from datasets import load_dataset, concatenate_datasets
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig, MarianMTModel
import torch.cuda
from torch import bfloat16

from typing import List, Dict, Tuple, Union
from tqdm import tqdm

import dotenv
import os
dotenv.load_dotenv(dotenv_path='../.env')

True

In [12]:
HF_TOKEN = os.getenv('HF_DS_TOKEN')

In [13]:
 # 'MultiNLI_Dutch_translated_with_Marianmt' 
 # 'SNLI_Dutch_translated_with_Marianmt'
 
DS_NAME =  'Apollo_Dutch_translated_with_NLLB200'

In [ ]:
'''
base = "medalpaca"
sub = ["medical_meadow_mediqa",
      #"medical_meadow_usmle_self_assessment",
      # "medical_meadow_mmmlu",
      # "medical_meadow_medical_flashcards",
      # "medical_meadow_wikidoc_patient_information",
      # "medical_meadow_wikidoc",
      # "medical_meadow_pubmed_causal",
      # "medical_meadow_medqa",
      # "medical_meadow_health_advice",
      # "medical_meadow_cord19"
      ]
MedAlpaca_sets = [load_dataset(f"{base}/{s}", split="train") for s in sub]
MedAlpaca = concatenate_datasets(MedAlpaca_sets)
'''

In [17]:
# integer-label triplet
DS = load_dataset('FreedomIntelligence/ApolloCorpus')  # multi_nli, ('ms_marco', 'v2.1')

ValueError: Invalid pattern: '**' can only be an entire path component

In [ ]:
# facebook/nllb-200-3.3B
# facebook/m2m100_418M
# facebook/mbart-large-50-many-to-many-mmt
# t5-large
# DeepL API
# vvn/en-to-dutch-marianmt

manual_dmap = False
MODEL = "facebook/nllb-200-3.3B"

if manual_dmap:
    from accelerate import infer_auto_device_map, init_empty_weights
    config = AutoConfig.from_pretrained(MODEL)
    with init_empty_weights():
        zero_model = AutoModelForSeq2SeqLM.from_config(config)
    device_map = infer_auto_device_map(zero_model,
                                    max_memory={0: "2GiB", 1: "2GiB", 2: "2GiB", 3: "2GiB", "cpu": "20GiB"},
                                    no_split_module_classes=[])

tokenizer = AutoTokenizer.from_pretrained(MODEL)
ntm_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL,
                                                  device_map='auto')
#ntm_model.to(device_map)
# # vvn/en-to-dutch-marianmt FremyCompany/opus-mt-nl-en-healthcare

device = "cuda:0" if (torch.cuda.is_available()) & (torch.cuda.device_count()==1) else "cpu"
if torch.cuda.device_count()<=1:
    if device =='cuda:0':
        #ntm_model.half()
        ntm_model.to(device)
        ntm_model.eval()

In [ ]:
num_token = []
for r in tqdm(DS['validation_matched']):
    num_tokens = len(tokenizer(r['premise'], 
                               r['hypothesis'], 
                               return_tensors='pt')['input_ids'][0])
    num_token.append(num_tokens)

In [ ]:
# from eng_XX to nl_XX, or from eng_Latn to nld_Latn

def get_translations(BATCH, device='cuda:0', multilingual=False):
    tokens = tokenizer(list(BATCH), return_tensors="pt", padding=True, truncation=True).to(device)
    if multilingual:
        outputs = ntm_model.generate(**tokens, forced_bos_token_id=tokenizer.lang_code_to_id["nl_XX"], max_length=512)
    else:
        outputs = ntm_model.generate(**tokens, max_length=512)    
    translated_sentences = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return [t.replace("▁", " ").strip() for t in translated_sentences]

In [ ]:
def Relevance(txt: str) -> bool:
    # check if string is relevant
    return True

def Process(txts: List[Tuple[str]], tuple_len=2)->List[Tuple[str]]:
    # filter
    #for t in txts:
      # check if string is relevant
      #if Relevance(t)==False:
      #  return None
    # translate        
    assert(tuple_len in [2,3]), "tuple_len must be 2 or 3"
    
    # TODO, make this for abitrarily long tuples
    part_1 = get_translations([t[0] for t in txts], multilingual=False)
    part_2 = get_translations([t[1] for t in txts], multilingual=False)
    
    if tuple_len == 3:
        part_3 = get_translations([t[2] for t in txts], multilingual=False)
    
    return list(zip(part_1, part_2, part_3)) if tuple_len == 3 else list(zip(part_1, part_2))

def datasetParser(dataset, text_keys=['premise', 'hypothesis'], pass_keys=['label'], batch_size=32, test=True):
    dList = [] 
    batch = []
    labels =[]
    
    for d in tqdm(dataset):
        # make batches
        if batch.__len__() == batch_size:
            processed = Process(txts = batch, tuple_len=len(text_keys))                               
            for i, _processed in enumerate(processed):
                tdict = {text_keys[j]: s for j,s in enumerate(_processed)}
                
                for k in pass_keys:
                    tdict[k] = labels[i][pass_keys.index(k)]
                      
                dList.append(tdict)
            batch = []
            labels = []
            if test:
                return dList
        else:
            batch.append(tuple([d[k] for k in text_keys]))
            labels.append(tuple([d[k] for k in pass_keys]))
    return dList

In [ ]:
if "keys" in DS.__dir__():
        print(f"Processing {len(DS.keys())} datasets")
        for k in DS.keys():
                print(f"Processing {k}, with {DS[k].num_rows} rows")
        FinalDict = {k: Dataset.from_list(datasetParser(DS[k], 
                                                        text_keys=TEXT_KEYS, 
                                                        pass_keys=PASS_KEYS,
                                                        batch_size=128))
                        for k in DS.keys()
                }
else:
        FinalDict = {"train": Dataset.from_list(datasetParser(DS, 
                                                              text_keys=TEXT_KEYS, 
                                                              pass_keys=PASS_KEYS,
                                                              batch_size=128))}
        
FinalDataset = DatasetDict(FinalDict)
FinalDataset.push_to_hub(f"UMCU/{DS_NAME}", token=HF_TOKEN, private=True)